In [1]:
import os
import glob
import tarfile
import requests
import numpy as np
import cudf


In [2]:
# Set environment variable to only see the selected GPU 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
parent_dir = "/".join(os.getcwd().split("/")[:-1])

In [4]:
url_dict =    {
    "airline-data.2005-2015.tar.gz" : "https://diybigdata.net/?sdm_process_download=1&download_id=392",
    "airline-data.2003-2018.tar.gz" : "https://diybigdata.net/?sdm_process_download=1&download_id=821",
    "airline-data.1998-2018.tar.gz" : "https://diybigdata.net/?sdm_process_download=1&download_id=832"}


In [5]:

def download_data(key):
    url = url_dict[key]
    # Send GET request to the link
    response = requests.get(url, stream=True)

    # Check if request was successful
    if response.status_code == 200:
        # Get filename from response headers (assuming Content-Disposition header is present)
        filename = response.headers.get('Content-Disposition')
        if filename:
          filename = filename.split("=")[1].strip('"')
        else:
          # If filename unavailable, use a key as filename
          filename = key

        # Open a file for writing in binary mode
        with open(f"{parent_dir}/data/{filename}", "wb") as f:
          for chunk in response.iter_content(1024):
            # Write downloaded data in chunks
            f.write(chunk)
        print(f"File downloaded successfully: {filename}")
    else:
        print(f"Download failed with status code: {response.status_code}")

In [6]:
def extract_data(fname):
    if fname.endswith("tar.gz"):
        tar = tarfile.open(fname, "r:gz")
        tar.extractall(filter="data", path=f"{parent_dir}/data/")
        tar.close()
    elif fname.endswith("tar"):
        tar = tarfile.open(fname, "r:")
        tar.extractall(filter="data", path=f"{parent_dir}/data/")
        tar.close()

In [9]:
def extract_small_data():

    # Define archive filename
    archive_filename = f"{parent_dir}/data/airline-data.1998-2018.tar.gz"

    # Open archive in read mode
    try:
        with tarfile.open(archive_filename, "r:gz") as tar:
            # Extract all filenames
            all_filenames = tar.getnames()
            # print (all_filenames)

            # Filter files starting with "On_Time_On_Time_Performance_2005"
            filtered_files = [f for f in all_filenames if f.split("/")[-1].startswith("On_Time_On_Time_Performance_2003")]

            if filtered_files:
                print("Extracting files:")
                for member in filtered_files:
                # Extract member (file) to output directory (if provided)
                    tar.extract(member, filter="data" ,path=f"{parent_dir}/data/") 
                    print(f"- {member}")  # Print extracted filename
            else:
                print("No files starting with 'On_Time_On_Time_Performance_2003' found")
    except FileNotFoundError:
        print(f"Error: Archive '{archive_filename}' not found")


In [10]:
def prepare_dataset(use_full_dataset=False):
    all_data_frames = []

    if use_full_dataset:
        if os.path.isfile (f"{parent_dir}/data/airline-data.1998-2018.tar.gz" )== False:
            download_data(key="airline-data.1998-2018.tar.gz")
            extract_data(fname=f"{parent_dir}/data/airline-data.1998-2018.tar.gz")
        else:
            extract_data(fname=f"{parent_dir}/data/airline-data.1998-2018.tar.gz")

        for m in glob.glob(
            f"{parent_dir}/data/airline-data/On_Time_On_Time_Performance*.csv"
        ):
            all_data_frames.append(cudf.read_csv(m))
    else:
        if os.path.isfile( f"{parent_dir}/data/airline-data.1998-2018.tar.gz") == False:
            download_data(key="airline-data.1998-2018.tar.gz")
            extract_small_data()
        else : 
            extract_small_data()

        for m in glob.glob(
            f"{parent_dir}/data/airline-data/On_Time_On_Time_Performance_2003_*.csv"
        ):
            all_data_frames.append(cudf.read_csv(m))

    # colect all the dataframes
    dataset = cudf.concat(all_data_frames)

    # pass all column names to Capital
    capital_names = [x.upper() for x in dataset.columns.to_list() ]
    dataset.columns = capital_names
    print (capital_names)


    # define the features 
    input_cols = ['YEAR','MONTH','DAY_OF_MONTH','DAY_OF_WEEK','CRS_DEP_TIME','CRS_ARR_TIME',
                  'OP_UNIQUE_CARRIER','OP_CARRIER_FL_NUM','ACTUAL_ELAPSED_TIME','ORIGIN',
                  'DEST','DISTANCE','DIVERTED']

    # encode categoricals as numeric
    for col in dataset.select_dtypes(["object"]).columns:
        dataset[col] = dataset[col].astype("category").cat.codes.astype(np.int32)

    # cast all columns to int32
    for col in dataset.columns:
        dataset[col] = dataset[col].astype(np.float32)  # needed for random forest

    # define the label
    dataset["ArrDelayBinary"] = 1.0 * (dataset["ARR_DELAY"] > 10)

    # put target/label column first [ classic XGBoost standard ]
    output_cols = ["ArrDelayBinary"] + input_cols

    # select the columns of interest
    dataset = dataset[output_cols]

    # drop the nan values
    dataset.dropna(axis=0, inplace=True)

    dataset = dataset.reindex(columns=output_cols)
    # dataset.to_orc(orc_name)
    return dataset

In [11]:
df = prepare_dataset()

['./airline-data', './airline-data/On_Time_On_Time_Performance_2005_10.csv', './airline-data/On_Time_On_Time_Performance_2005_12.csv', './airline-data/On_Time_On_Time_Performance_1998_5.csv', './airline-data/On_Time_On_Time_Performance_2005_2.csv', './airline-data/On_Time_On_Time_Performance_1998_8.csv', './airline-data/On_Time_On_Time_Performance_2005_5.csv', './airline-data/On_Time_On_Time_Performance_2005_8.csv', './airline-data/On_Time_On_Time_Performance_1999_10.csv', './airline-data/On_Time_On_Time_Performance_2006_1.csv', './airline-data/On_Time_On_Time_Performance_2018_2.csv', './airline-data/On_Time_On_Time_Performance_1999_2.csv', './airline-data/On_Time_On_Time_Performance_2006_11.csv', './airline-data/On_Time_On_Time_Performance_2006_12.csv', './airline-data/On_Time_On_Time_Performance_2006_4.csv', './airline-data/On_Time_On_Time_Performance_2000_10.csv', './airline-data/On_Time_On_Time_Performance_2006_5.csv', './airline-data/LUT-DOT_airline_IDs.csv', './airline-data/On_Ti

In [12]:
df.describe()

/home/barradd/miniconda3/envs/rapids-23.12/lib/python3.10/site-packages/cudf/core/dataframe.py:5059: FutureWarning: `datetime_is_numeric` is deprecated. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  warnings.warn(
/home/barradd/miniconda3/envs/rapids-23.12/lib/python3.10/site-packages/cudf/core/series.py:3334: FutureWarning: `datetime_is_numeric` is deprecated and will be removed in a future release. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  warnings.warn(


,ArrDelayBinary,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,CRS_DEP_TIME,CRS_ARR_TIME,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ACTUAL_ELAPSED_TIME,ORIGIN,DEST,DISTANCE,DIVERTED
count,6.375689e+06,6.375689e+06,6.375689e+06,6.375689e+06,6.375689e+06,6.375689e+06,6.375689e+06,6.375689e+06,6.375689e+06,6.375689e+06,6.375689e+06,6.375689e+06,6.375689e+06,6375689.0
mean,2.053390e-01,2.003000e+03,6.532758e+00,1.573585e+01,3.934335e+00,5.774579e+02,8.061868e+02,9.294674e+00,2.814078e+03,1.223457e+02,1.415992e+02,1.414844e+02,7.136091e+02,0.0
std,4.039490e-01,0.000000e+00,3.445176e+00,8.787332e+00,1.988135e+00,2.735536e+02,2.798882e+02,5.681457e+00,2.124432e+03,6.925841e+01,7.626457e+01,7.615202e+01,5.650007e+02,0.0
min,0.000000e+00,2.003000e+03,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-6.900000e+02,0.000000e+00,0.000000e+00,1.200000e+01,0.0
25%,0.000000e+00,2.003000e+03,4.000000e+00,8.000000e+00,2.000000e+00,3.380000e+02,5.780000e+02,5.000000e+00,8.700000e+02,7.100000e+01,7.600000e+01,7.600000e+01,3.040000e+02,0.0
50%,0.000000e+00,2.003000e+03,7.000000e+00,1.600000e+01,4.000000e+00,5.720000e+02,8.180000e+02,1.000000e+01,2.430000e+03,1.030000e+02,1.490000e+02,1.490000e+02,5.500000e+02,0.0
75%,0.000000e+00,2.003000e+03,1.000000e+01,2.300000e+01,6.000000e+00,8.070000e+02,1.038000e+03,1.500000e+01,4.511000e+03,1.530000e+02,2.060000e+02,2.050000e+02,9.480000e+02,0.0
max,1.000000e+00,2.003000e+03,1.200000e+01,3.100000e+01,7.000000e+00,1.190000e+03,1.329000e+03,1.700000e+01,6.920000e+03,1.233000e+03,2.810000e+02,2.810000e+02,4.962000e+03,0.0


In [ ]:
6375689